# Load Data

In [1]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")
sample_sub_df = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")

# Exploratory Data Analysis

In [3]:
print("Original Number of Rows: ", len(train_df))
train_df = train_df.dropna()
print("Filtered Number of Rows: ", len(train_df))

Original Number of Rows:  750000
Filtered Number of Rows:  750000


In [4]:
train_df.groupby('Fertilizer Name')['id'].count()

Fertilizer Name
10-26-26    113887
14-35-14    114436
17-17-17    112453
20-20       110889
28-28       111158
DAP          94860
Urea         92317
Name: id, dtype: int64

In [5]:
fertilizer_column = 'Fertilizer Name'
numerical_features = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
for feature in numerical_features:
    print(f"\n--- Distribution of '{feature}' for each '{fertilizer_column}' ---")
    distribution = train_df.groupby(fertilizer_column)[feature].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
    print(distribution)


--- Distribution of 'Temparature' for each 'Fertilizer Name' ---
                      mean  median       std  min  max   count
Fertilizer Name                                               
10-26-26         31.470589    32.0  4.038993   25   38  113887
14-35-14         31.543247    32.0  4.001384   25   38  114436
17-17-17         31.463803    31.0  4.030426   25   38  112453
20-20            31.514406    31.0  4.025341   25   38  110889
28-28            31.518973    32.0  4.029370   25   38  111158
DAP              31.508065    32.0  4.051078   25   38   94860
Urea             31.507296    32.0  4.001744   25   38   92317

--- Distribution of 'Humidity' for each 'Fertilizer Name' ---
                      mean  median       std  min  max   count
Fertilizer Name                                               
10-26-26         60.937043    61.0  6.640019   50   72  113887
14-35-14         60.962669    61.0  6.641917   50   72  114436
17-17-17         60.998755    61.0  6.612568   50   

In [6]:
train_df['P/N Ratio'] = train_df['Potassium'] / train_df['Nitrogen']
train_df['K/N Ratio'] = train_df['Phosphorous'] / train_df['Nitrogen']
train_df['Env Max'] = train_df[['Temparature', 'Humidity', 'Moisture']].max(axis=1)
synthetic_features = ['P/N Ratio', 'K/N Ratio', 'Env Max']

for feature in synthetic_features:
    print(f"\n--- Distribution of '{feature}' for each '{fertilizer_column}' ---")
    train_df[feature] = train_df[feature].replace([np.inf, -np.inf], np.nan)
    distribution = train_df.groupby(fertilizer_column)[feature].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
    print(distribution)
    
test_df['P/N Ratio'] = test_df['Potassium'] / test_df['Nitrogen']
test_df['K/N Ratio'] = test_df['Phosphorous'] / test_df['Nitrogen']
test_df['Env Max'] = test_df[['Temparature', 'Humidity', 'Moisture']].max(axis=1)


--- Distribution of 'P/N Ratio' for each 'Fertilizer Name' ---
                     mean    median       std  min   max   count
Fertilizer Name                                                 
10-26-26         0.601993  0.411765  0.671760  0.0  4.75  113887
14-35-14         0.595436  0.411765  0.657911  0.0  4.75  114436
17-17-17         0.596140  0.409091  0.656708  0.0  4.75  112453
20-20            0.609874  0.416667  0.674232  0.0  4.75  110889
28-28            0.605453  0.411765  0.671077  0.0  4.75  111158
DAP              0.590018  0.400000  0.669557  0.0  4.75   94860
Urea             0.605761  0.407407  0.681578  0.0  4.75   92317

--- Distribution of 'K/N Ratio' for each 'Fertilizer Name' ---
                     mean    median       std  min   max   count
Fertilizer Name                                                 
10-26-26         1.337029  0.903226  1.493318  0.0  10.5  113887
14-35-14         1.347489  0.926829  1.465407  0.0  10.5  114436
17-17-17         1.318852  

## Mutual Infromation

In [7]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

X = train_df.drop(columns=['Fertilizer Name'])
y = train_df['Fertilizer Name']
# Identify categorical and numerical columns in X
categorical_cols_X = X.select_dtypes(include=['object']).columns
numerical_cols_X = X.select_dtypes(include=[np.number]).columns

# Apply Label Encoding to categorical features in X and the target variable y
le = LabelEncoder()
for col in categorical_cols_X:
    X[col] = le.fit_transform(X[col])
y_encoded = le.fit_transform(y)

# Create a boolean mask for discrete features for mutual_info_classif
discrete_features_mask = np.array([col in categorical_cols_X for col in X.columns])

# Calculate mutual information scores
mi_scores = mutual_info_classif(X, y_encoded, discrete_features=discrete_features_mask, random_state=42)

# Create a Series for better readability and sort them
mi_series = pd.Series(mi_scores, index=X.columns)
mi_series = mi_series.sort_values(ascending=False)

print("Mutual Information Scores with respect to 'Fertilizer Name':")
print(mi_series)

Mutual Information Scores with respect to 'Fertilizer Name':
K/N Ratio      0.008024
P/N Ratio      0.003845
Phosphorous    0.003201
Moisture       0.002858
Env Max        0.002267
Nitrogen       0.002263
Humidity       0.002151
Crop Type      0.002088
Potassium      0.002057
Temparature    0.001872
Soil Type      0.000617
id             0.000000
dtype: float64


# Training Model

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Define numerical features and target variable
numerical_features = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
categorical_features = ['Soil Type', 'Crop Type']
synthetic_features = ['P/N Ratio', 'K/N Ratio', 'Env Max']
target = 'Fertilizer Name'

# Extract features (X) and target (y)
X = train_df[numerical_features + categorical_features].copy()
y = train_df[target]

# add rank
for col in numerical_features:
    X.loc[:, f'{col}_Rank'] = X[col].rank(method='average', ascending=True)
    test_df.loc[:, f'{col}_Rank'] = test_df[col].rank(method='average', ascending=True)

# Preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

# Apply preprocessing to X
X_train = preprocessor.fit_transform(X)
X_test = preprocessor.transform(test_df[numerical_features + categorical_features])

# Encode the target variable y using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
fertilizer_classes = label_encoder.classes_
print("Fertilizer classes (order):", fertilizer_classes)

X_train = pd.DataFrame(np.array(X_train))
X_test = pd.DataFrame(np.array(X_test))
y_encoded = pd.DataFrame(np.array(y_encoded))

Fertilizer classes (order): ['10-26-26' '14-35-14' '17-17-17' '20-20' '28-28' 'DAP' 'Urea']


In [9]:
X_train = pd.DataFrame(np.array(X_train))
X_test = pd.DataFrame(np.array(X_test))
y_encoded = pd.DataFrame(np.array(y_encoded))

## Metric: Mean Average Precision @ 3 (MAP@3)

In [10]:
def mapk(actual, predicted, k=3):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def apk(actual, predicted, k=3):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

## Base Model: XGBoost & LightGBM

In [11]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier, log_evaluation, early_stopping

model_configs = {
    'xgb': {'model': XGBClassifier, 'params': {'objective': 'multi:softprob', 
                                               'num_class': y.nunique(), 
                                               'max_depth': 7, 
                                               'learning_rate': 0.03, 
                                               'subsample': 0.8, 
                                               'colsample_bytree': 0.4, 
                                               'tree_method': 'hist', 
                                               'random_state': 42, 
                                               'eval_metric': 'mlogloss', 
                                               'device': "cuda", 
                                               'n_estimators':10000,
                                               'early_stopping_rounds': 100}},
    'lgb': {'model': LGBMClassifier, 'params': {'objective': 'multiclass', 
                                                'num_class': y.nunique(), 
                                                "device": "gpu", 
                                                "learning_rate": 0.03, 
                                                "max_depth": 7, 
                                                "n_estimators": 10000, 
                                                "n_jobs": -1, 
                                                "num_leaves": 31, 
                                                "random_state": 42, 
                                                "verbose": -1}}
}

In [12]:
import warnings

# Suppress warnings as before
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

In [13]:
from sklearn.model_selection import StratifiedKFold

num_of_folds = 10
skf = StratifiedKFold(n_splits=num_of_folds, shuffle=True, random_state=42)

oof_preds = {name: np.zeros((len(X_train), y.nunique())) for name in model_configs}
test_preds = {name: np.zeros((len(X_test), y.nunique())) for name in model_configs}

for name, config in model_configs.items():
    print(f"\n Training Base Model: {name.upper()}")
    current_model_test_preds = np.zeros((len(X_test), y.nunique()))

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_encoded)):
        print(f"  Fold {fold + 1}/{num_of_folds}")

        x_train_fit = X_train.iloc[train_idx].values
        x_valid_fit = X_train.iloc[valid_idx].values
        y_train_fit = y_encoded.iloc[train_idx].values
        y_valid_fit = y_encoded.iloc[valid_idx].values
        
        model = config['model'](**config['params'])

        if name == 'xgb':
            model.fit(x_train_fit, y_train_fit,
                      eval_set=[(x_train_fit, y_train_fit), (x_valid_fit, y_valid_fit)],
                      verbose=100)
            
            if hasattr(model, 'evals_result_') and model.evals_result_:
                print(f"    XGBoost Fold {fold+1} Final Train Loss: {model.evals_result_['validation_0']['mlogloss'][-1]:.4f}")
                print(f"    XGBoost Fold {fold+1} Final Valid Loss: {model.evals_result_['validation_1']['mlogloss'][-1]:.4f}")
            else:
                print(f"    XGBoost Fold {fold+1}: No evaluation results found (check eval_set or verbose).")

        else: # For LightGBM
            # eval_set data is now also simply NumPy arrays
            eval_set_lgbm = [
                (x_train_fit, y_train_fit),
                (x_valid_fit, y_valid_fit)
            ]
            callbacks = [lgb.early_stopping(100, verbose=False), lgb.log_evaluation(period=100)]

            model.fit(x_train_fit, y_train_fit,
                      eval_set=eval_set_lgbm,
                      callbacks=callbacks)

            if model.best_iteration_:
                print(f"    LightGBM Fold {fold+1} Best Iteration: {model.best_iteration_}")
                print(f"    LightGBM Fold {fold+1} Final Valid Loss (Best Score): {model.best_score_['valid_1']['multi_logloss']:.4f}")
            else:
                print(f"    LightGBM Fold {fold+1} Final Valid Loss: {model.evals_result_['valid_1']['multi_logloss'][-1]:.4f}")
        
        valid_preds = model.predict_proba(x_valid_fit)
        test_preds_fold = model.predict_proba(X_test)

        oof_preds[name][valid_idx] = valid_preds
        current_model_test_preds += test_preds_fold / num_of_folds

    test_preds[name] = current_model_test_preds

print("\nBase model training complete for all configurations.")


 Training Base Model: XGB
  Fold 1/10
[0]	validation_0-mlogloss:1.94563	validation_1-mlogloss:1.94567
[100]	validation_0-mlogloss:1.92856	validation_1-mlogloss:1.93376
[200]	validation_0-mlogloss:1.91833	validation_1-mlogloss:1.92843
[300]	validation_0-mlogloss:1.91005	validation_1-mlogloss:1.92474
[400]	validation_0-mlogloss:1.90298	validation_1-mlogloss:1.92207
[500]	validation_0-mlogloss:1.89651	validation_1-mlogloss:1.91973
[600]	validation_0-mlogloss:1.89054	validation_1-mlogloss:1.91785
[700]	validation_0-mlogloss:1.88471	validation_1-mlogloss:1.91623
[800]	validation_0-mlogloss:1.87916	validation_1-mlogloss:1.91486
[900]	validation_0-mlogloss:1.87389	validation_1-mlogloss:1.91369
[1000]	validation_0-mlogloss:1.86886	validation_1-mlogloss:1.91255
[1100]	validation_0-mlogloss:1.86393	validation_1-mlogloss:1.91164
[1200]	validation_0-mlogloss:1.85929	validation_1-mlogloss:1.91081
[1300]	validation_0-mlogloss:1.85473	validation_1-mlogloss:1.91007
[1400]	validation_0-mlogloss:1.8504

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [16:31:38] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


  Fold 2/10
[0]	validation_0-mlogloss:1.94564	validation_1-mlogloss:1.94567
[100]	validation_0-mlogloss:1.92857	validation_1-mlogloss:1.93361
[200]	validation_0-mlogloss:1.91826	validation_1-mlogloss:1.92809
[300]	validation_0-mlogloss:1.91007	validation_1-mlogloss:1.92440
[400]	validation_0-mlogloss:1.90300	validation_1-mlogloss:1.92160
[500]	validation_0-mlogloss:1.89661	validation_1-mlogloss:1.91938
[600]	validation_0-mlogloss:1.89061	validation_1-mlogloss:1.91750
[700]	validation_0-mlogloss:1.88476	validation_1-mlogloss:1.91583
[800]	validation_0-mlogloss:1.87924	validation_1-mlogloss:1.91438
[900]	validation_0-mlogloss:1.87396	validation_1-mlogloss:1.91313
[1000]	validation_0-mlogloss:1.86897	validation_1-mlogloss:1.91210
[1100]	validation_0-mlogloss:1.86406	validation_1-mlogloss:1.91104
[1200]	validation_0-mlogloss:1.85944	validation_1-mlogloss:1.91010
[1300]	validation_0-mlogloss:1.85486	validation_1-mlogloss:1.90926
[1400]	validation_0-mlogloss:1.85051	validation_1-mlogloss:1.9

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[100]	training's multi_logloss: 1.92734	valid_1's multi_logloss: 1.93177
[200]	training's multi_logloss: 1.91892	valid_1's multi_logloss: 1.92751
[300]	training's multi_logloss: 1.91244	valid_1's multi_logloss: 1.92529
[400]	training's multi_logloss: 1.90672	valid_1's multi_logloss: 1.92384
[500]	training's multi_logloss: 1.90135	valid_1's multi_logloss: 1.92267
[600]	training's multi_logloss: 1.89626	valid_1's multi_logloss: 1.92159
[700]	training's multi_logloss: 1.89147	valid_1's multi_logloss: 1.92084
[800]	training's multi_logloss: 1.88694	valid_1's multi_logloss: 1.92026
[900]	training's multi_logloss: 1.88252	valid_1's multi_logloss: 1.91966
[1000]	training's multi_logloss: 1.87828	valid_1's multi_logloss: 1.91923
[1100]	training's multi_logloss: 1.8742	valid_1's multi_logloss: 1.91893
[1200]	training's multi_logloss: 1.87018	valid_1's multi_logloss: 1.91857
[1300]	training's multi_logloss: 1.86622	valid_1's multi_logloss: 1.91834
[1400]	training's multi_logloss: 1.86239	valid_1

...

[100]	training's multi_logloss: 1.92733	valid_1's multi_logloss: 1.93202
[200]	training's multi_logloss: 1.91881	valid_1's multi_logloss: 1.92775
[300]	training's multi_logloss: 1.91234	valid_1's multi_logloss: 1.92561
[400]	training's multi_logloss: 1.90661	valid_1's multi_logloss: 1.92406
[500]	training's multi_logloss: 1.90127	valid_1's multi_logloss: 1.92273
[600]	training's multi_logloss: 1.89626	valid_1's multi_logloss: 1.92184
[700]	training's multi_logloss: 1.8915	valid_1's multi_logloss: 1.92115
[800]	training's multi_logloss: 1.88687	valid_1's multi_logloss: 1.92041
[900]	training's multi_logloss: 1.88251	valid_1's multi_logloss: 1.91987
[1000]	training's multi_logloss: 1.8783	valid_1's multi_logloss: 1.91943
[1100]	training's multi_logloss: 1.87425	valid_1's multi_logloss: 1.91912
[1200]	training's multi_logloss: 1.87019	valid_1's multi_logloss: 1.91881
[1300]	training's multi_logloss: 1.8662	valid_1's multi_logloss: 1.91847
[1400]	training's multi_logloss: 1.86229	valid_1's

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[100]	training's multi_logloss: 1.92746	valid_1's multi_logloss: 1.93175
[200]	training's multi_logloss: 1.91902	valid_1's multi_logloss: 1.92728
[300]	training's multi_logloss: 1.91251	valid_1's multi_logloss: 1.92484
[400]	training's multi_logloss: 1.90676	valid_1's multi_logloss: 1.92324
[500]	training's multi_logloss: 1.90144	valid_1's multi_logloss: 1.92201
[600]	training's multi_logloss: 1.89641	valid_1's multi_logloss: 1.92092
[700]	training's multi_logloss: 1.89154	valid_1's multi_logloss: 1.91996
[800]	training's multi_logloss: 1.88694	valid_1's multi_logloss: 1.91923
[900]	training's multi_logloss: 1.88257	valid_1's multi_logloss: 1.91867
[1000]	training's multi_logloss: 1.8783	valid_1's multi_logloss: 1.91814
[1100]	training's multi_logloss: 1.87419	valid_1's multi_logloss: 1.91775
[1200]	training's multi_logloss: 1.87021	valid_1's multi_logloss: 1.91737
[1300]	training's multi_logloss: 1.86626	valid_1's multi_logloss: 1.91702
[1400]	training's multi_logloss: 1.86243	valid_1

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[100]	training's multi_logloss: 1.92732	valid_1's multi_logloss: 1.93194
[200]	training's multi_logloss: 1.91885	valid_1's multi_logloss: 1.9277
[300]	training's multi_logloss: 1.9124	valid_1's multi_logloss: 1.92551
[400]	training's multi_logloss: 1.90658	valid_1's multi_logloss: 1.92397
[500]	training's multi_logloss: 1.90125	valid_1's multi_logloss: 1.92271
[600]	training's multi_logloss: 1.89619	valid_1's multi_logloss: 1.92175
[700]	training's multi_logloss: 1.89144	valid_1's multi_logloss: 1.92103
[800]	training's multi_logloss: 1.88696	valid_1's multi_logloss: 1.92043
[900]	training's multi_logloss: 1.88254	valid_1's multi_logloss: 1.91995
[1000]	training's multi_logloss: 1.87828	valid_1's multi_logloss: 1.9195
[1100]	training's multi_logloss: 1.87413	valid_1's multi_logloss: 1.9191
[1200]	training's multi_logloss: 1.87007	valid_1's multi_logloss: 1.91873
[1300]	training's multi_logloss: 1.86606	valid_1's multi_logloss: 1.91849
[1400]	training's multi_logloss: 1.86222	valid_1's 

In [14]:
test_preds

{'xgb': array([[0.17704039, 0.10883566, 0.11805922, ..., 0.15576426, 0.17522261,
         0.11778907],
        [0.13562677, 0.08898067, 0.25316377, ..., 0.11801846, 0.094445  ,
         0.15641103],
        [0.13479239, 0.12679926, 0.13260986, ..., 0.16841967, 0.12499098,
         0.12070203],
        ...,
        [0.15255471, 0.17119473, 0.1258512 , ..., 0.10221327, 0.18734015,
         0.13161796],
        [0.23351015, 0.10626551, 0.1479719 , ..., 0.18762837, 0.13454856,
         0.12081731],
        [0.16092811, 0.23132355, 0.16813521, ..., 0.10829261, 0.05643388,
         0.08873593]]),
 'lgb': array([[0.13660293, 0.12659007, 0.12768463, ..., 0.16593347, 0.16303582,
         0.13298996],
        [0.15100126, 0.11534047, 0.20895447, ..., 0.14009263, 0.09200501,
         0.12340266],
        [0.14494362, 0.14921054, 0.11588994, ..., 0.15326092, 0.10337601,
         0.15110053],
        ...,
        [0.13530365, 0.18744408, 0.12066773, ..., 0.11022424, 0.17223096,
         0.15016077]

## Stacking Ensemble Meta Model: LightGBM

In [15]:
stacking_train_feats = np.hstack([oof_preds[name] for name in oof_preds])
stacking_test_feats = np.hstack([test_preds[name] for name in test_preds])
meta_model = LGBMClassifier(objective='multiclass', num_class=y.nunique(), 
                            learning_rate=0.03, n_estimators=1000, 
                            random_state=42, verbose=-1, device='gpu')
final_test_preds = np.zeros((len(X_test), y.nunique()))
for fold, (train_idx, valid_idx) in enumerate(skf.split(stacking_train_feats, y)):
    meta_model.fit(stacking_train_feats[train_idx], y_encoded.iloc[train_idx], 
                   eval_set=[(stacking_train_feats[valid_idx], y_encoded.iloc[valid_idx])], 
                   callbacks=[lgb.early_stopping(100, verbose=False)])
    final_test_preds += meta_model.predict_proba(stacking_test_feats) / num_of_folds

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was e

# Make Prediction

In [16]:
top_3_preds_indices = np.argsort(final_test_preds, axis=1)[:, ::-1][:, :3]
label_encoder.fit(train_df['Fertilizer Name'].unique()) # Re-fit on original labels
top_3_labels = label_encoder.inverse_transform(top_3_preds_indices.ravel()).reshape(top_3_preds_indices.shape)
submission = pd.DataFrame({
    'id': test_df['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('submission.csv', index=False)
print("Successfully generated 'submission.csv'")
print(submission.head())

Successfully generated 'submission.csv'
       id       Fertilizer Name
0  750000    DAP 10-26-26 28-28
1  750001   17-17-17 Urea 20-20
2  750002  20-20 28-28 10-26-26
3  750003     14-35-14 DAP Urea
4  750004   20-20 10-26-26 Urea
